In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
import json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def draw_hist_boxplot(df,features_names):
  sns.boxplot(x=features_names, data = df)
  sns.stripplot(x=features_names, data = df, color = '#474646')


def replace_outliers_with_median(df, feature_name):
              Q1 = df[feature_name].quantile(0.25)
              median = df[feature_name].quantile(0.5)
              Q3 = df[feature_name].quantile(0.75)
              IQR = Q3 - Q1
              lower_whisker = Q1 - 1.5 * IQR
              upper_whisker = Q3 + 1.5 * IQR
              df[feature_name] = np.where((df[feature_name] < lower_whisker) |
                                          (df[feature_name] > upper_whisker),
                                          median,
                                          df[feature_name])
            
def columns_without_values(data):
    columns = data.isnull().sum()
    print(columns[columns > 0])

In [ ]:
df_numbers = pd.read_json('/content/drive/MyDrive/result.json')

In [ ]:
df_numbers.drop(df_numbers[df_numbers.year < 2010].index, inplace = True)

In [ ]:
df_numbers.drop(df_numbers[df_numbers.year > 2020].index, inplace = True)

In [ ]:
df_numbers = df_numbers.drop(['fuelType','make','transmissionType', 'model', 'province','id','brand','brand_id', 'model_id', 'color', 'title', 'seller_type', 'fuel_types', 'bodyType', 'provinceId'], axis = 1)

In [ ]:
target = 'price'
all_features = ['fuelTypeId', 'km', 'makeId', 'modelId','transmissionTypeId', 'year', 'cubicCapacity', 'doors', 'hp']

In [ ]:
X_train_full, X_test = train_test_split(df_numbers[all_features + [target]], random_state=1)
X_train, X_val = train_test_split(X_train_full, random_state=1)
y_train = X_train[target].values
y_val = X_val[target].values
y_test = X_test[target].values
X_train.drop([target], axis=1, inplace=True)
X_val.drop([target], axis=1, inplace=True)
X_test.drop([target], axis=1, inplace=True)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(all_features)]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                640       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________


In [ ]:
example_result = model.predict(X_test)
example_result

array([[ 4.86579688e+04],
       [ 2.99259238e+04],
       [ 1.15586094e+04],
       [ 2.16620820e+04],
       [ 1.53599121e+04],
       [ 4.24214697e+03],
       [ 2.67819531e+04],
       [ 1.34599307e+04],
       [ 1.38172041e+04],
       [-8.01885107e+03],
       [ 1.18986455e+04],
       [ 3.63961987e+03],
       [ 1.90690566e+04],
       [ 1.42978584e+04],
       [ 1.06908838e+04],
       [ 4.11174658e+03],
       [ 1.62396797e+04],
       [ 3.01914277e+04],
       [ 1.91542598e+04],
       [ 2.06395176e+04],
       [ 9.49442578e+03],
       [ 1.56293584e+04],
       [ 2.14739375e+04],
       [ 5.33717578e+03],
       [ 2.58674941e+04],
       [ 6.35650098e+03],
       [ 9.50732812e+03],
       [ 1.47338027e+04],
       [ 1.38763662e+04],
       [ 3.22046855e+04],
       [ 2.44320801e+04],
       [ 1.39282393e+04],
       [ 2.36369043e+04],
       [ 1.52614561e+04],
       [ 2.14397480e+04],
       [ 2.27695566e+04],
       [ 8.51033594e+03],
       [ 1.54713145e+04],
       [ 8.2

In [ ]:
model.fit(X_train, y_train, epochs=10,validation_split = 0.2, verbose=0)

In [ ]:
model.save('keras.h5')